In [2]:
%defaultDatasource jdbc:h2:mem:db

# Analisando o Uso de Medicamentos
* Esta tarefa analisará os dados conforme três abordagens: análise do comportamento estatístico; análise de correlação entre pares; análise de correlação de variáveis.
* A partir dos arquivos CSV que estão no diretórios /data/nhanes considere as seguintes tabelas SQL.

In [3]:
CREATE TABLE Pessoa (
  Id VARCHAR(5) NOT NULL,
  Genero INTEGER,
  Idade INTEGER,
  PRIMARY KEY(Id)
) AS SELECT
    Id,
    Genero,
    Idade
FROM CSVREAD('../../../data/nhanes/demographic-person.csv');

CREATE TABLE Medicamento (
  Codigo VARCHAR(6) NOT NULL,
  NomeMedicamento VARCHAR(100) NOT NULL,
  PRIMARY KEY(Codigo)
) AS SELECT
    Codigo,
    NomeMedicamento
FROM CSVREAD('../../../data/nhanes/medications-drug.csv');

CREATE TABLE UsoMedicamento (
  IdPessoa VARCHAR(5) NOT NULL,
  CodMedicamento VARCHAR(6) NOT NULL,
  DiasUso INTEGER,
  FOREIGN KEY(IdPessoa)
    REFERENCES Pessoa(Id)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
  FOREIGN KEY(CodMedicamento)
    REFERENCES Medicamento(Codigo)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT
    IdPessoa,
    CodMedicamento,
    DiasUso
FROM CSVREAD('../../../data/nhanes/medications-use.csv');

In [4]:
SELECT * FROM Pessoa;
SELECT * FROM Medicamento;
SELECT * FROM UsoMedicamento;

# I. Análise de comportamento estatístico

## 1) Contabilizando o uso de medicamentos

* Liste o nome de todos os medicamentos e a quantidade de pessoas que usa cada um deles.

In [5]:
SELECT M.NomeMedicamento, COUNT(U.IdPessoa)
    FROM Medicamento M, UsoMedicamento U, Pessoa P
    WHERE M.Codigo = U.CodMedicamento AND U.IdPessoa = P.Id
    GROUP BY M.NomeMedicamento

## 2) Medicamento mais usado com VIEW

* Informe o código do medicamento mais usado: fazendo uso de VIEW.

In [6]:
CREATE VIEW MaisUsado AS
    SELECT COUNT(U.IdPessoa) Contagem, M.NomeMedicamento Nome
    FROM Medicamento M, UsoMedicamento U, Pessoa P
    WHERE M.Codigo = U.CodMedicamento AND U.IdPessoa = P.Id
    GROUP BY M.NomeMedicamento;

In [7]:
SELECT MAX(MU.Contagem) FROM MaisUsado MU;

493

## 3) Medicamento mais usado sem VIEW

* Informe o código do medicamento mais usado: sem fazer uso de VIEW.

In [8]:
SELECT TOP 1 Contagem.Codigo
    FROM (
        SELECT COUNT(U.IdPessoa) Pessoas, M.Codigo Codigo
        FROM Medicamento M, UsoMedicamento U, Pessoa P
        WHERE M.Codigo = U.CodMedicamento AND U.IdPessoa = P.Id
        GROUP BY M.Codigo) Contagem
    ORDER BY Contagem.Pessoas DESC

d00732

## 4) Nome do medicamento mais usado
* Informe o nome do medicamento mais usado (uso de VIEW é opcional).

In [9]:
SELECT MU.Nome
    FROM MaisUsado MU
    WHERE MU.Contagem IN (SELECT MAX(MU.Contagem) FROM MaisUsado MU);

LISINOPRIL

## 5) Contabilizando quanto as pessoas usam de medicamento

* Informe o número médio de uso de medicamento por pessoa.

In [10]:
SELECT AVG(Meds.Contagem)
    FROM (SELECT P.Id, COUNT(U.CodMedicamento) Contagem
            FROM UsoMedicamento U, Pessoa P, Medicamento M
            WHERE U.IdPessoa = P.Id AND M.Codigo = U.CodMedicamento
            GROUP BY P.Id) Meds

3

## 6) Pessoas que usam mais do que a média

* Liste o id das pessoas que usam mais medicamentos do que a média.

In [11]:
SELECT Meds.Id
    FROM (SELECT P.Id Id, COUNT(U.CodMedicamento) Contagem
            FROM UsoMedicamento U, Pessoa P, Medicamento M
            WHERE U.IdPessoa = P.Id AND M.Codigo = U.CodMedicamento
            GROUP BY P.Id) Meds
    WHERE 3 < Meds.Contagem

## 7) Análise do uso de medicamento

* Considere que um médico quer fazer responder a seguinte questão: pessoas tendem a usar mais medicamentos conforme ficam mais velhas?
* Escreva uma query que ajude o médico a realizar esta análise.

In [15]:
SELECT AVG(P.Idade)
    FROM (SELECT Meds.Id
            FROM (SELECT P.Id Id, COUNT(U.CodMedicamento) Contagem
                    FROM UsoMedicamento U, Pessoa P, Medicamento M
                    WHERE U.IdPessoa = P.Id AND M.Codigo = U.CodMedicamento
                    GROUP BY P.Id) Meds
            WHERE 3 < Meds.Contagem) Media, Pessoa P
    WHERE P.Id = Media.Id

61

# II. Análise de correlação em pares
## Estudo de ação cruzada entre medicamentos

* Um pesquisador quer estudar o efeito cruzado de usar dois medicamentos simultaneamente. 
* Para isso ele precisará da seguinte sequência de queries.

## 8) Medicamentos tomados em conjunto
* Construa uma VIEW com duas colunas: medicamentoA, medicamentoB
* Nessa VIEW, é criada uma tupla (medicamentoA, medicamentoB) toda vez que uma pessoa toma o medicamento A e também o medicamento B.
* Haverá repetição de tuplas, já que mais de uma pessoa pode tomar o mesmo par de medicamentos.
* Cuidado para não duplicar a mesma informação, ou seja, considerando que uma pessoa tomou o medicamentoX e o medicamentoY, cuidado para não criar duas tuplas: (medicamentoX, medicamentoY) e (medicamentoY, medicamentoX).

In [ ]:
CREATE VIEW MedicamentoX AS
    SELECT U.IdPessoa PessoaA, U.CodMedicamento MedA
    FROM UsoMedicamento U;
    
CREATE VIEW MedicamentoConjunto AS
    SELECT U.CodMedicamento MedB, MX.MedA MedA
    FROM UsoMedicamento U, MedicamentoX MX
    WHERE U.IdPessoa = MX.PessoaA AND U.CodMedicamento > MX.MedA;
    
SELECT * FROM MedicamentoConjunto;

## 9) Total de medicamento tomados em conjunto
* Apresente o total de pessoas que toma cada par de medicamentos.

In [ ]:
SELECT MC.MedA, MC.MedB, COUNT(*)
    FROM MedicamentoConjunto MC
    GROUP BY MC.MedA, MC.MedB;

# III. Análise de correlação entre variáveis
* Consiste na análise da seguinte questão: como a alteração de uma variável afeta a outra.
## Matriz de análise
* Considere a matriz abaixo.

In [ ]:
CREATE TABLE Matriz (
  Id VARCHAR(5) NOT NULL,
  Genero INTEGER,
  Idade INTEGER,
  IdadeAte60 BOOLEAN,
  MaisUmAnoMedicamento BOOLEAN,
  MedicamentosAcimaMedia BOOLEAN,
  PRIMARY KEY(Id)
) AS SELECT
    Id,
    Genero,
    Idade,
    FALSE, FALSE, FALSE
FROM CSVREAD('../../../data/nhanes/demographic-person.csv');

SELECT * FROM Matriz;

## 10) Atualização simples da matriz
* Utilize o comando UPDATE para atualizar a coluna IdadeAte60 da tabela colocando verdadeiro para as pessoas que têm até 60 anos de idade.

In [ ]:
UPDATE Matriz
    SET IdadeAte60 = TRUE
    WHERE Idade <=60;

SELECT * FROM Matriz;

## 11) Atualização vinculada a uma segunda tabela
* Utilize o comando UPDATE para atualizar a coluna MaisUmAnoMedicamento, colocando verdadeiro para aquelas pessoas que usam pelo menos um medicamento há mais de um ano.

In [ ]:
UPDATE Matriz
    SET MaisUmAnoMedicamento = TRUE
    WHERE Id IN (SELECT DISTINCT U.IdPessoa
        FROM UsoMedicamento U
        WHERE U.DiasUso > 365);
             
SELECT * FROM Matriz;

## 12) Atualização baseada em uma consulta complexa
* Utilize o comando UPDATE para atualizar a coluna MedicamentosAcimaMedia, colocando verdadeiro para aquelas pessoas que usam um número de medicamentos acima da média.

## 13) Que análise interessante pode ser feita?
* A partir da Matriz criada, proponha uma análise interessante.

### 13a) Descreva a seguir (em Markdown) qual a análise que você propõe.

### 13b) Escreva uma consulta em SQL que fornece informações para a análise proposta.